##### You are revisiting the Healthcare for All Case Study. You are provided with this historical data about Donors and how much they donated. Your task is to build a machine learning model that will help the company identify people who are more likely to donate and then try to predict the donation amount.

### Import the required libraries and modules that you would need.



In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

In [2]:
categorical = pd.read_csv('categorical.csv')
numerical = pd.read_csv('numerical.csv')
targets = pd.read_csv('target.csv')


In [3]:
targets.dtypes

TARGET_B      int64
TARGET_D    float64
dtype: object

In [4]:
display(categorical.info())
display(numerical.info())
targets.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95412 entries, 0 to 95411
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   STATE         95412 non-null  object 
 1   CLUSTER       95412 non-null  int64  
 2   HOMEOWNR      95412 non-null  object 
 3   GENDER        95412 non-null  object 
 4   DATASRCE      95412 non-null  int64  
 5   RFA_2R        95412 non-null  object 
 6   RFA_2A        95412 non-null  object 
 7   GEOCODE2      95412 non-null  object 
 8   DOMAIN_A      95412 non-null  object 
 9   DOMAIN_B      95412 non-null  int64  
 10  ODATEW_YR     95412 non-null  int64  
 11  ODATEW_MM     95412 non-null  int64  
 12  DOB_YR        95412 non-null  int64  
 13  DOB_MM        95412 non-null  int64  
 14  MINRDATE_YR   95412 non-null  int64  
 15  MINRDATE_MM   95412 non-null  int64  
 16  MAXRDATE_YR   95412 non-null  int64  
 17  MAXRDATE_MM   95412 non-null  int64  
 18  LASTDATE_YR   95412 non-nu

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95412 entries, 0 to 95411
Columns: 315 entries, TCODE to CLUSTER2
dtypes: float64(9), int64(306)
memory usage: 229.3 MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95412 entries, 0 to 95411
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   TARGET_B  95412 non-null  int64  
 1   TARGET_D  95412 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 1.5 MB


In [5]:
numerical.head()

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,WEALTH2,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,CARDPROM,NUMPROM,CARDPM12,NUMPRM12,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2
0,0,60.000000,5,9,0,0,39,34,18,10,2,1,5,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,0.0,177.0,682.0,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,27,74,6,14,240.0,31,14,5.0,12.0,10.0,4,7.741935,95515,0,4,39
1,1,46.000000,6,9,16,0,15,55,11,6,2,1,9,3611,940,998,99,0,0,50,50,67,0,0,31,6,4,2,6,4,14,0,0,2,0,1,4,34,41,43,32,42,45,32,33,46,21,13,14,33,23,10,4,2,11,16,36,22,15,12,1,5,4,21,75,55,23,9,69,4,3,24,317,360,99,99,0,0,0,0,0,0,0,5468,5218,12,10,96,4,97,3,9,59,94,88,55,95,5,4,1,3,5,4,2,18,44,5,0,0,0,97,98,98,98,99,94,0,83,76,73,21,5,0,0,0,4,0,0,0,91,91,91,94,4480.0,13.0,803.0,1088,1096,1026,1037,36175,2,6,2,5,15,14,13,10,33,2,5,2,5,15,14,14,10,32,6,2,66,3,56,44,9,80,14,0,0,0,0,0,0,6,0,2,24,32,12,71,70,83,58,81,57,64,57,99,99,0,22,24,4,21,13,2,1,6,0,4,1,0,3,1,0,6,13,1,2,8,18,11,4,3,4,10,7,11,1,6,2,1,16,69,5,2,160,5,5,12,21,7,30,20,14,24,4,24,10,0,0,0,8,15,0,55,10,11,0,0,2,0,3,1,1,2,3,1,1,0,3,0,0,0,42,39,50,7,27,16,99,92,53,5,10,2,26,56,97,99,0,0,0,96,0,4,0,0,0,99,0,99,99,99,20,4,6,5,12,32,6,13,47.0,3,1,10.0,25.0,25.0,18,15.666667,148535,0,2,1
2,1,61.611649,3,1,2,0,20,29,33,6,8,1,1,7001,2040,2669,0,2,98,49,51,96,2,0,0,2,0,0,0,0,0,0,0,2,0,0,0,35,43,46,37,45,49,23,35,40,25,13,20,19,16,13,10,8,15,14,30,22,19,25,10,23,21,35,44,22,6,2,63,9,9,19,183,254,69,69,1,6,5,3,3,3,0,497,546,2,1,78,22,93,7,18,36,76,65,30,86,14,7,2,5,11,17,3,17,60,18,0,1,0,0,1,6,18,50,0,4,36,49,51,14,5,4,2,24,11,2,3,6,0,2,9,44,0.0,281.0,518.0,251,292,292,340,11576,32,18,20,15,12,2,0,0,1,20,19,24,18,16,2,0,0,1,28,8,31,11,38,62,8,74,22,0,0,0,0,0,2,2,1,21,19,24,6,61,65,73,59,70,56,78,62,82,99,4,10,5,2,6,12,0,1,9,5,18,20,5,7,6,0,11,33,4,3,2,12,3,3

In [6]:
for col in numerical.columns:
    print(col, numerical[col].value_counts())

TCODE 0        40917
1        25691
2        17006
28        8258
1002      1861
3          835
4          367
28028       71
72          63
980         54
13          45
4002        44
14          22
18          17
116         17
45          17
42          15
22          13
24          11
39002       11
13002        7
30           7
228          5
6            5
23           5
136          5
202          4
21           3
94           2
134          2
17           2
6400         2
9            2
14002        2
4004         2
100          2
18002        2
7            1
44           1
24002        1
93           1
27           1
50           1
76           1
96           1
38           1
58002        1
12           1
72002        1
25           1
40           1
36           1
47           1
61           1
22002        1
Name: TCODE, dtype: int64
AGE 61.611649    23665
50.000000     1930
76.000000     1885
72.000000     1813
68.000000     1809
             ...  
9.000000         1
6.0000

HV1 0       1117
675      262
550      211
875      187
425      180
        ... 
5729       1
5319       1
5472       1
3915       1
3347       1
Name: HV1, Length: 4434, dtype: int64
HV2 0       1117
625      146
547      138
642      136
571      135
        ... 
177        1
4187       1
4580       1
5605       1
5680       1
Name: HV2, Length: 4623, dtype: int64
HV3 3     21368
4     17948
2     17878
5     12466
6      7939
7      5124
1      3173
8      2860
9      2182
12     1297
10     1082
11      999
0       904
13      192
Name: HV3, dtype: int64
HV4 2     18818
3     18473
4     16117
5     11334
1      9676
6      7374
7      4960
8      3256
9      1945
10     1344
0      1229
11      697
12      144
13       45
Name: HV4, dtype: int64
HU1 84    2670
82    2638
85    2544
81    2530
83    2491
      ... 
3      124
4      110
5      110
6       94
2       94
Name: HU1, Length: 100, dtype: int64
HU2 16    2669
18    2652
15    2542
19    2531
14    2484
      ... 
91    

LFC2 78    3724
76    3687
80    3670
79    3655
77    3621
      ... 
5       10
2        5
4        5
1        2
3        2
Name: LFC2, Length: 100, dtype: int64
LFC3 58    3387
57    3304
59    3302
56    3187
55    3147
      ... 
95       8
94       6
96       5
97       5
98       1
Name: LFC3, Length: 99, dtype: int64
LFC4 75    3464
76    3436
74    3313
77    3309
73    3292
      ... 
5       20
4        5
2        2
1        2
3        2
Name: LFC4, Length: 100, dtype: int64
LFC5 55    3190
53    3171
54    3145
56    3099
57    3057
      ... 
93      11
96       5
94       4
95       3
97       1
Name: LFC5, Length: 98, dtype: int64
LFC6 68    3152
69    3094
65    3076
67    2972
70    2955
      ... 
7        5
6        4
5        2
4        1
3        1
Name: LFC6, Length: 98, dtype: int64
LFC7 55    2704
52    2659
50    2638
53    2600
54    2535
      ... 
2       20
96      17
1        8
97       8
98       1
Name: LFC7, Length: 100, dtype: int64
LFC8 99    28256
0 

AVGGIFT 15.000000    6020
20.000000    4566
25.000000    2659
10.000000    2227
12.500000    1800
             ... 
12.548387       1
3.146341        1
14.402222       1
2.409091        1
96.794872       1
Name: AVGGIFT, Length: 7713, dtype: int64
CONTROLN 95515     1
16713     1
142025    1
1290      1
119028    1
         ..
77198     1
109598    1
17943     1
141702    1
185114    1
Name: CONTROLN, Length: 95412, dtype: int64
HPHONE_D 1    47765
0    47647
Name: HPHONE_D, dtype: int64
RFA_2F 1    47675
2    20545
3    15291
4    11901
Name: RFA_2F, dtype: int64
CLUSTER2 13    3466
5     3154
57    2666
59    2653
15    2559
      ... 
30     647
46     644
29     569
40     369
6      211
Name: CLUSTER2, Length: 62, dtype: int64


In [7]:
nulls_percent_df = pd.DataFrame(numerical.isna().sum()/len(numerical)).reset_index()
nulls_percent_df.columns = ['column_name', 'nulls_percentage']

In [8]:
nulls_percent_df

,column_name,nulls_percentage
0,TCODE,0.0
1,AGE,0.0
2,INCOME,0.0
3,WEALTH1,0.0
4,HIT,0.0
...,...,...
310,AVGGIFT,0.0
311,CONTROLN,0.0
312,HPHONE_D,0.0
313,RFA_2F,0.0


In [9]:
nulls_percent_df[nulls_percent_df['nulls_percentage']!=0]

,column_name,nulls_percentage


In [10]:
#No null values seen

In [11]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
numerical_scaled = scaler.fit_transform(numerical)

In [12]:
pd.DataFrame(numerical_scaled).head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314
0,0.000000,0.608247,0.666667,1.000000,0.000000,0.000000,0.393939,0.343434,0.181818,0.101010,0.020202,0.011494,0.555556,0.010051,0.011108,0.009378,0.0,0.353535,0.656566,0.474747,0.535354,0.929293,0.010101,0.0,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.113402,0.0,0.000000,0.000000,0.464286,0.571429,0.607143,0.476190,0.595238,0.642857,0.333333,0.313131,0.424242,0.272727,0.111111,0.141414,0.181818,0.171717,0.131313,0.111111,0.151515,0.121212,0.111111,0.343434,0.252525,0.181818,0.262626,0.101010,0.232323,0.181818,0.333333,0.494949,0.282828,0.121212,0.040404,0.616162,0.070707,0.164384,0.191919,0.304615,0.394286,0.979798,0.959596,0.020202,0.020202,0.000000,0.000000,0.070707,0.070707,0.000000,0.079833,0.105833,0.230769,0.153846,0.868687,0.141414,0.969697,0.040404,0.070707,0.383838,0.808081,0.707071,0.323232,0.848485,0.161616,0.060606,0.04,0.050505,0.090909,0.151515,0.030303,0.226667,0.505051,0.252525,0.000000,0.000000,0.000000,0.020202,0.070707,0.131313,0.272727,0.474747,0.000000,0.010101,0.616162,0.682353,0.677778,0.245902,0.100,0.020202,0.000000,0.000000,0.141414,0.010101,0.000000,0.000000,0.020202,0.050505,0.171717,0.737374,0.000000,0.271889,0.774120,0.204667,0.212000,0.232667,0.252000,0.073818,0.131313,0.232323,0.232323,0.232323,0.151515,0.010101,0.00,0.000000,0.010101,0.040404,0.252525,0.242424,0.262626,0.171717,0.020202,0.00,0.00000,0.020202,0.282828,0.040404,0.515152,0.010101,0.464646,0.545455,0.030303,0.888889,0.080808,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.040404,0.010101,0.131313,0.155556,0.210526,0.020202,0.454545,0.565657,0.646465,0.505051,0.646465,0.444444,0.626263,0.535354,1.000000,0.0,0.000000,0.090909,0.030303,0.080808,0.131313,0.090909,0.000000,0.054545,0.090909,0.030303,0.151515,0.191919,0.050505,0.040404,0.030303,0.0,0.030303,0.414141,0.010101,0.000000,0.070707,0.131313,0.060606,0.050505,0.000000,0.059701,0.090909,0.040404,0.010101,0.030303,0.101010,0.020202,0.010101,0.070707,0.787879,0.020202,0.000000,0.705882,0.161616,0.101010,0.393939,0.212121,0.216216,0.040404,0.030303,0.051546,0.202020,0.100000,0.263889,0.040404,0.000000,0.000000,0.000000,0.181818,0.393939,0.000000,0.343434,0.232323,0.181818,0.161616,0.012048,0.040404,0.000000,0.250000,0.000000,0.000000,0.050505,0.018182,0.000000,0.000000,0.000000,0.000000,0.00,0.074074,0.0,0.030303,0.747475,0.888889,0.080808,0.000000,0.040404,0.969697,0.777778,0.191919,0.419355,0.596154,0.050505,0.141414,0.141414,0.313131,0.545455,0.464646,0.0,0.0,0.909091,0.000000,0.101010,0.000000,0.0,0.000000,0.333333,0.656566,0.404040,1.000000,1.000000,0.285714,0.4,0.101010,0.070707,0.433333,0.366492,0.315789,0.168831,0.023965,0.127119,0.341463,0.005,0.001401,0.010,0.003676,0.006465,0.498045,0.0,1.000000,0.622951
1,0.000014,0.463918,0.833333,1.000000,0.066390,0.000000,0.151515,0.555556,0.111111,0.060606,0.020202,0.011494,1.000000

In [13]:
from sklearn.feature_selection import VarianceThreshold 
var_threshold = 0.02
sel = VarianceThreshold(threshold=(var_threshold))

# 1) default is threshold = 0 :eliminate all coumns that are identical for all rows
# 2) in practice we would scale the columns first, and then apply threshold, or apply 
#    a different threshold for different columns

# This drops the columns that have a variance less than this threshold
sel = sel.fit(numerical_scaled)
temp = sel.transform(numerical_scaled)
temp = pd.DataFrame(temp)
print(numerical_scaled.shape)
print(temp.shape)

(95412, 315)
(95412, 79)


In [14]:
pd.DataFrame(temp).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78
0,0.608247,0.666667,1.000000,0.343434,0.181818,0.555556,0.0,0.353535,0.656566,0.929293,0.010101,0.494949,0.979798,0.959596,0.020202,0.000000,0.000000,0.079833,0.105833,0.230769,0.153846,0.868687,0.141414,0.070707,0.808081,0.707071,0.848485,0.505051,0.020202,0.070707,0.131313,0.272727,0.474747,0.000000,0.616162,0.000000,0.000000,0.000000,0.020202,0.050505,0.171717,0.737374,0.000000,0.271889,0.131313,0.515152,0.464646,0.545455,0.131313,0.454545,0.646465,0.646465,0.626263,0.535354,1.000000,0.0,0.343434,0.181818,0.747475,0.888889,0.777778,0.596154,0.141414,0.141414,0.313131,0.545455,0.464646,0.909091,0.101010,0.333333,0.656566,0.404040,0.285714,0.4,0.433333,0.498045,0.0,1.000000,0.622951
1,0.463918,0.833333,1.000000,0.555556,0.111111,1.000000,1.0,0.000000,0.000000,0.676768,0.000000,0.757576,1.000000,1.000000,0.000000,0.000000,0.000000,0.911333,0.869667,0.923077,0.769231,0.969697,0.040404,0.090909,0.949495,0.888889,0.959596,0.444444,0.979798,0.989899,0.989899,0.989899,1.000000,0.949495,0.838384,0.000000,0.000000,0.000000,0.919192,0.919192,0.919192,0.949495,0.478632,0.019969,0.020202,0.666667,0.565657,0.444444,0.020202,0.717172,0.838384,0.818182,0.646465,0.575758,1.000000,1.0,0.555556,0.111111,0.393939,0.505051,0.929293,0.192308,0.262626,0.565657,0.979798,1.000000,0.000000,0.969697,0.040404,1.000000,0.000000,1.000000,0.952381,0.8,0.183333,0.774510,0.0,0.333333,0.000000
2,0.624862,0.333333,0.111111,0.292929,0.333333,0.111111,0.0,0.020202,0.989899,0.969697,0.020202,0.444444,0.696970,0.696970,0.060606,0.050505,0.030303,0.082833,0.091000,0.153846,0.076923,0.787879,0.222222,0.181818,0.767677,0.656566,0.868687,0.606061,0.000000,0.010101,0.060606,0.181818,0.505051,0.000000,0.363636,0.020202,0.242424,0.030303,0.000000,0.020202,0.090909,0.444444,0.000000,0.431644,0.323232,0.313131,0.383838,0.626263,0.212121,0.616162,0.737374,0.707071,0.787879,0.626263,0.828283,1.0,0.282828,0.323232,0.848485,0.969697,0.656566,0.423077,0.121212,0.232323,0.505051,0.696970,0.313131,0.000000,0.353535,0.222222,0.777778,0.171717,0.428571,0.4,0.416667,0.078617,1.0,1.000000,0.967213
3,0.711340,0.000000,0.444444,0.141414,0.313131,0.000000,0.0,0.080808,0.929293,0.616162,0.000000,0.505051,0.858586,0.838384,0.040404,0.010101,0.000000,0.166667,0.210500,0.153846,0.076923,0.484848,0.525253,0.060606,0.737374,0.616162,0.848485,0.363636,0.101010,0.252525,0.505051,0.696970,0.929293,0.101010,0.424242,0.000000,0.090909,0.000000,0.010101,0.080808,0.171717,0.343434,0.997863,0.102919,0.272727,0.262626,0.464646,0.545455,0.171717,0.434343,0.818182,0.686869,0.333333,0.313131,0.000000,1.0,0.141414,0.313131,0.676768,0.565657,0.434343,0.730769,0.101010,0.191919,0.393939,0.454545,0.555556,0.454545,0.171717,0.232323,0.777778,0.222222,0.761905,0.4,0.433333,0.899764,1.0,1.000000,0.655738
4,0.793814,0.333333,0.222222,0.090909,0.535354,1.000000,1.0,0.000000,0.000000,0.020202,0.989899,0.575758,1.000000,0.989899,0.000000,0.000000,0.000000,0.096000,0.099000,0.307692,0.230769,0.909091,0.101010,0.000000,0.828283,0.494949,0.929293,0.010101,0.000000,0.010101,0.020202,0.161616,0.676768,0.000000,0.454545,0.000000,0.000000,0.000000,0.252525,0.585859,0.747475,0.838384,0.534188,0.195084,0.242424,0.111111,0.202020,0.808081,0.010101,0.767677,0.737374,0.656566,0.808081,0.313131,0.818182,1.0,0.090909,0.535354,0.656566,1.000000,0.454545,0.653846,0.010101,0.030303,0.060606,0.333333,0.676768,0.090909,0.727273,1.000000,0.010101,0.212121,0.285714,0.4,0.700000,0.037079,1.0,0.333333,0.409836


In [15]:
# To check which columns were removed, you can manipulate the results
# from the code below. This gives the variance of each feature in order
# of appearance of the dataset.
sel.variances_


array([1.75493674e-04, 2.21932083e-02, 8.02275908e-02, 9.03764075e-02,
       1.49132173e-03, 2.61802339e-03, 1.34243682e-02, 2.32567811e-02,
       3.19975201e-02, 1.96784193e-03, 2.68155697e-03, 2.28104662e-03,
       1.02613600e-01, 3.38581196e-03, 3.76408939e-03, 3.59628980e-03,
       2.29533182e-01, 9.94600474e-02, 1.63556065e-01, 3.16042737e-03,
       3.39492297e-03, 4.50544144e-02, 2.84235197e-02, 1.22804261e-03,
       5.09957894e-03, 1.93912343e-02, 9.53325826e-04, 9.61133151e-04,
       6.55013636e-04, 1.18787514e-03, 4.75926813e-04, 5.03014665e-04,
       2.63911441e-04, 1.36515733e-02, 5.14425138e-04, 1.56917030e-03,
       1.53123652e-03, 9.84719265e-03, 9.64517465e-03, 9.31928327e-03,
       7.47231939e-03, 6.86524606e-03, 6.71358129e-03, 1.00407315e-02,
       6.91452469e-03, 4.15375273e-03, 3.46457167e-03, 3.75706547e-03,
       6.18802365e-03, 3.93599421e-03, 1.90088942e-03, 1.75339900e-03,
       3.67776591e-03, 4.61258144e-03, 2.67556157e-03, 1.39422327e-03,
      

In [16]:
#This gives you the result as True and False for the columns that we
# selected and those which were removed, respectively.
sel.variances_ > var_threshold
sel.get_support()
var_list = list(sel.get_support())

In [17]:
var_list

[False,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 False,
 True,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 True,
 False,
 True,

In [18]:
list(zip(numerical.columns, var_list))

[('TCODE', False),
 ('AGE', True),
 ('INCOME', True),
 ('WEALTH1', True),
 ('HIT', False),
 ('MALEMILI', False),
 ('MALEVET', False),
 ('VIETVETS', True),
 ('WWIIVETS', True),
 ('LOCALGOV', False),
 ('STATEGOV', False),
 ('FEDGOV', False),
 ('WEALTH2', True),
 ('POP901', False),
 ('POP902', False),
 ('POP903', False),
 ('POP90C1', True),
 ('POP90C2', True),
 ('POP90C3', True),
 ('POP90C4', False),
 ('POP90C5', False),
 ('ETH1', True),
 ('ETH2', True),
 ('ETH3', False),
 ('ETH4', False),
 ('ETH5', False),
 ('ETH6', False),
 ('ETH7', False),
 ('ETH8', False),
 ('ETH9', False),
 ('ETH10', False),
 ('ETH11', False),
 ('ETH12', False),
 ('ETH13', False),
 ('ETH14', False),
 ('ETH15', False),
 ('ETH16', False),
 ('AGE901', False),
 ('AGE902', False),
 ('AGE903', False),
 ('AGE904', False),
 ('AGE905', False),
 ('AGE906', False),
 ('AGE907', False),
 ('CHIL1', False),
 ('CHIL2', False),
 ('CHIL3', False),
 ('AGEC1', False),
 ('AGEC2', False),
 ('AGEC3', False),
 ('AGEC4', False),
 ('AGEC5', F

In [19]:
[col[0] for col in zip(numerical.columns, var_list) if col[1] == False]

['TCODE',
 'HIT',
 'MALEMILI',
 'MALEVET',
 'LOCALGOV',
 'STATEGOV',
 'FEDGOV',
 'POP901',
 'POP902',
 'POP903',
 'POP90C4',
 'POP90C5',
 'ETH3',
 'ETH4',
 'ETH5',
 'ETH6',
 'ETH7',
 'ETH8',
 'ETH9',
 'ETH10',
 'ETH11',
 'ETH12',
 'ETH13',
 'ETH14',
 'ETH15',
 'ETH16',
 'AGE901',
 'AGE902',
 'AGE903',
 'AGE904',
 'AGE905',
 'AGE906',
 'AGE907',
 'CHIL1',
 'CHIL2',
 'CHIL3',
 'AGEC1',
 'AGEC2',
 'AGEC3',
 'AGEC4',
 'AGEC5',
 'AGEC6',
 'AGEC7',
 'CHILC1',
 'CHILC2',
 'CHILC3',
 'CHILC4',
 'CHILC5',
 'HHAGE1',
 'HHAGE2',
 'HHAGE3',
 'HHN1',
 'HHN2',
 'HHN4',
 'HHN5',
 'HHN6',
 'MARR1',
 'MARR2',
 'MARR3',
 'MARR4',
 'HHP1',
 'HHP2',
 'DW3',
 'DW7',
 'DW8',
 'DW9',
 'HU3',
 'HU4',
 'HHD1',
 'HHD4',
 'HHD6',
 'HHD7',
 'HHD8',
 'HHD9',
 'HHD10',
 'HHD11',
 'HHD12',
 'ETHC1',
 'ETHC3',
 'ETHC4',
 'ETHC5',
 'ETHC6',
 'HUR1',
 'RHP1',
 'RHP2',
 'RHP3',
 'RHP4',
 'HUPA1',
 'HUPA4',
 'HUPA5',
 'HUPA7',
 'DMA',
 'IC1',
 'IC2',
 'IC3',
 'IC4',
 'IC5',
 'IC7',
 'IC8',
 'IC9',
 'IC10',
 'IC11',
 'IC1

In [20]:
drop_columns = ['TCODE',
 'HIT',
 'MALEMILI',
 'MALEVET',
 'LOCALGOV',
 'STATEGOV',
 'FEDGOV',
 'POP901',
 'POP902',
 'POP903',
 'POP90C4',
 'POP90C5',
 'ETH3',
 'ETH4',
 'ETH5',
 'ETH6',
 'ETH7',
 'ETH8',
 'ETH9',
 'ETH10',
 'ETH11',
 'ETH12',
 'ETH13',
 'ETH14',
 'ETH15',
 'ETH16',
 'AGE901',
 'AGE902',
 'AGE903',
 'AGE904',
 'AGE905',
 'AGE906',
 'AGE907',
 'CHIL1',
 'CHIL2',
 'CHIL3',
 'AGEC1',
 'AGEC2',
 'AGEC3',
 'AGEC4',
 'AGEC5',
 'AGEC6',
 'AGEC7',
 'CHILC1',
 'CHILC2',
 'CHILC3',
 'CHILC4',
 'CHILC5',
 'HHAGE1',
 'HHAGE2',
 'HHAGE3',
 'HHN1',
 'HHN2',
 'HHN4',
 'HHN5',
 'HHN6',
 'MARR1',
 'MARR2',
 'MARR3',
 'MARR4',
 'HHP1',
 'HHP2',
 'DW3',
 'DW7',
 'DW8',
 'DW9',
 'HU3',
 'HU4',
 'HHD1',
 'HHD4',
 'HHD6',
 'HHD7',
 'HHD8',
 'HHD9',
 'HHD10',
 'HHD11',
 'HHD12',
 'ETHC1',
 'ETHC3',
 'ETHC4',
 'ETHC5',
 'ETHC6',
 'HUR1',
 'RHP1',
 'RHP2',
 'RHP3',
 'RHP4',
 'HUPA1',
 'HUPA4',
 'HUPA5',
 'HUPA7',
 'DMA',
 'IC1',
 'IC2',
 'IC3',
 'IC4',
 'IC5',
 'IC7',
 'IC8',
 'IC9',
 'IC10',
 'IC11',
 'IC12',
 'IC13',
 'IC14',
 'IC15',
 'IC16',
 'IC17',
 'IC18',
 'IC19',
 'IC20',
 'IC21',
 'IC22',
 'IC23',
 'HHAS1',
 'HHAS2',
 'HHAS4',
 'MC3',
 'TPE1',
 'TPE2',
 'TPE3',
 'TPE4',
 'TPE5',
 'TPE6',
 'TPE7',
 'TPE8',
 'TPE9',
 'PEC1',
 'TPE10',
 'TPE11',
 'TPE12',
 'LFC1',
 'LFC3',
 'LFC5',
 'LFC10',
 'OCC1',
 'OCC2',
 'OCC3',
 'OCC4',
 'OCC5',
 'OCC6',
 'OCC7',
 'OCC8',
 'OCC9',
 'OCC10',
 'OCC11',
 'OCC12',
 'OCC13',
 'EIC1',
 'EIC2',
 'EIC3',
 'EIC4',
 'EIC5',
 'EIC6',
 'EIC7',
 'EIC8',
 'EIC9',
 'EIC10',
 'EIC11',
 'EIC12',
 'EIC13',
 'EIC14',
 'EIC15',
 'EIC16',
 'OEDC1',
 'OEDC2',
 'OEDC3',
 'OEDC4',
 'OEDC5',
 'OEDC6',
 'OEDC7',
 'EC1',
 'EC2',
 'EC3',
 'EC4',
 'EC5',
 'EC6',
 'EC7',
 'EC8',
 'SEC1',
 'SEC2',
 'SEC3',
 'SEC4',
 'SEC5',
 'AFC1',
 'AFC2',
 'AFC3',
 'AFC4',
 'AFC5',
 'AFC6',
 'VC2',
 'VC4',
 'ANC1',
 'ANC2',
 'ANC3',
 'ANC4',
 'ANC5',
 'ANC6',
 'ANC7',
 'ANC8',
 'ANC9',
 'ANC10',
 'ANC11',
 'ANC12',
 'ANC13',
 'ANC14',
 'ANC15',
 'POBC1',
 'LSC2',
 'LSC3',
 'LSC4',
 'VOC1',
 'VOC3',
 'HC1',
 'HC3',
 'HC9',
 'HC10',
 'HC12',
 'HC14',
 'HC15',
 'HC16',
 'HC20',
 'HC21',
 'AC1',
 'AC2',
 'NUMPROM',
 'CARDPM12',
 'NUMPRM12',
 'RAMNTALL',
 'NGIFTALL',
 'CARDGIFT',
 'MINRAMNT',
 'MAXRAMNT',
 'LASTGIFT',
 'TIMELAG',
 'AVGGIFT']

In [21]:
num_clean = numerical.drop(columns=drop_columns, axis=1)


In [22]:
num_clean

,AGE,INCOME,WEALTH1,VIETVETS,WWIIVETS,WEALTH2,POP90C1,POP90C2,POP90C3,ETH1,ETH2,HHN3,DW1,DW2,DW4,DW5,DW6,HV1,HV2,HV3,HV4,HU1,HU2,HU5,HHD2,HHD3,HHD5,ETHC2,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR2,HUPA2,HUPA3,HUPA6,RP1,RP2,RP3,RP4,MSA,ADI,IC6,HHAS3,MC1,MC2,PEC2,TPE13,LFC2,LFC4,LFC6,LFC7,LFC8,LFC9,VC1,VC3,POBC2,LSC1,VOC2,HC2,HC4,HC5,HC6,HC7,HC8,HC11,HC13,HC17,HC18,HC19,MHUC1,MHUC2,CARDPROM,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2
0,60.000000,5,9,34,18,5,0,35,65,92,1,49,97,95,2,0,0,479,635,3,2,86,14,7,80,70,84,50,2,7,13,27,47,0,61,0,0,0,2,5,17,73,0.0,177.0,13,51,46,54,13,45,64,64,62,53,99,0,34,18,74,88,77,31,14,14,31,54,46,90,10,33,65,40,6,2,27,95515,0,4,39
1,46.000000,6,9,55,11,9,99,0,0,67,0,75,99,99,0,0,0,5468,5218,12,10,96,4,9,94,88,95,44,97,98,98,98,99,94,83,0,0,0,91,91,91,94,4480.0,13.0,2,66,56,44,2,71,83,81,64,57,99,99,55,11,39,50,92,10,26,56,97,99,0,96,4,99,0,99,20,4,12,148535,0,2,1
2,61.611649,3,1,29,33,1,0,2,98,96,2,44,69,69,6,5,3,497,546,2,1,78,22,18,76,65,86,60,0,1,6,18,50,0,36,2,24,3,0,2,9,44,0.0,281.0,32,31,38,62,21,61,73,70,78,62,82,99,28,32,84,96,65,22,12,23,50,69,31,0,35,22,77,17,9,2,26,15078,1,4,60
3,70.000000,1,4,14,31,0,0,8,92,61,0,50,85,83,4,1,0,1000,1263,2,1,48,52,6,73,61,84,36,10,25,50,69,92,10,42,0,9,0,1,8,17,34,9340.0,67.0,27,26,46,54,17,43,81,68,33,31,0,99,14,31,67,56,43,38,10,19,39,45,55,45,17,23,77,22,16,2,27,172556,1,4,41
4,78.000000,3,2,9,53,9,99,0,0,2,98,57,99,98,0,0,0,576,594,4,3,90,10,0,82,49,92,1,0,1,2,16,67,0,45,0,0,0,25,58,74,83,5000.0,127.0,24,11,20,80,1,76,73,65,80,31,81,99,9,53,65,99,45,34,1,3,6,33,67,9,72,99,1,21,6,2,43,7112,1,2,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,61.611649,5,9,47,11,9,99,0,0,78,10,48,54,38,32,24,14,988,1025,6,6,56,44,3,72,56,83,55,2,10,49,73,92,0,40,8,13,12,69,84,92,97,380.0,0.0,11,70,63,37,2,69,89,83,69,57,61,94,48,11,26,92,60,14,7,32,78,91,9,86,12,93,7,98,16,4,6,184568,0,1,12
95408,48.000000,7,9,43,19,9,96,0,4,91,3,70,89,88,1,0,0,1679,1723,3,3,88,12,0,89,85,96,58,18,71,88,91,97,5,77,0,10,0,16,26,44,79,3360.0,201.0,8,61,83,17,3,61,88,87,48,43,99,0,43,19,51,94,84,7,55,90,94,94,6,82,16,69,31,67,18,5,4,122706,1,1,2
95409,60.000000,5,9,46,20,9,99,0,0,82,14,44,96,95,2,1,0,376,377,4,3,66,34,10,64,43,80,49,0,0,0,1,9,0,45,0,0,0,3,34,78,91,4040.0,61.0,21,26,43,57,25,69,70,70,77,24,62,0,46,20,82,92,42,51,0,0,0,0,99,97,0,99,0,99,5,2,14,189641,1,3,34
95410,58.000000,7,9,35,20,7,99,0,0,92,1,65,99,99,0,0,0,2421,2459,11,10,88,12,0,85,71,84,56,89,96,99,99,99,9,90,0,0,0,88,88,90,91,8735.0,13.0,7,46,43,57,24,52,88,85,70,54,99,99,35,20,50,83,85,25,0,6,17,99,1,99,1,99,0,99,12,3,36,4693,1,4,11


In [23]:
categorical.head()


,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,ODATEW_YR,ODATEW_MM,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM
0,IL,36,H,F,3,L,E,C,T,2,89,1,37,12,92,8,94,2,95,12,89,11.0
1,CA,14,H,M,3,L,G,A,S,1,94,1,52,2,93,10,95,12,95,12,93,10.0
2,NC,43,U,M,3,L,E,C,R,2,90,1,0,2,91,11,92,7,95,12,90,1.0
3,CA,44,U,F,3,L,E,C,R,2,87,1,28,1,87,11,94,11,95,12,87,2.0
4,FL,16,H,F,3,L,F,A,S,2,86,1,20,1,93,10,96,1,96,1,79,3.0


In [24]:
categorical.isna().sum()/len(categorical)

STATE           0.000000
CLUSTER         0.000000
HOMEOWNR        0.000000
GENDER          0.000000
DATASRCE        0.000000
RFA_2R          0.000000
RFA_2A          0.000000
GEOCODE2        0.000000
DOMAIN_A        0.000000
DOMAIN_B        0.000000
ODATEW_YR       0.000000
ODATEW_MM       0.000000
DOB_YR          0.000000
DOB_MM          0.000000
MINRDATE_YR     0.000000
MINRDATE_MM     0.000000
MAXRDATE_YR     0.000000
MAXRDATE_MM     0.000000
LASTDATE_YR     0.000000
LASTDATE_MM     0.000000
FIRSTDATE_YR    0.000000
FIRSTDATE_MM    0.000021
dtype: float64

In [25]:
for col in categorical.columns:
    print(categorical[col].value_counts(dropna=False))

other    30457
CA       17343
FL        8376
TX        7535
IL        6420
MI        5654
NC        4160
WA        3577
GA        3403
IN        2980
WI        2795
MO        2712
Name: STATE, dtype: int64
40    6295
35    3624
36    3617
27    3553
24    3539
49    3315
12    3000
18    2897
13    2706
30    2607
39    2601
45    2515
43    2385
11    2378
51    2322
14    2246
41    2176
44    1953
16    1947
2     1942
21    1891
8     1828
10    1804
46    1800
28    1630
17    1621
20    1568
53    1441
3     1428
34    1358
42    1358
23    1324
31    1285
22    1282
25    1275
38    1200
15    1173
1     1140
7     1076
5     1014
37     972
26     947
47     868
29     859
32     795
48     787
50     776
9      770
6      603
33     593
4      561
19     497
52     270
Name: CLUSTER, dtype: int64
H    74582
U    20830
Name: HOMEOWNR, dtype: int64
F        54234
M        39094
other     2084
Name: GENDER, dtype: int64
3    64829
2    23455
1     7128
Name: DATASRCE, dtype: int6

In [26]:
#categorical columns to drop
# CLUSTER - one column and one unnamed column
# ODATEW_MM
# DOB_YR / MM 
# MINRDATE_YR 
# MAXRDATE YR 
# LASTDATE_YR
# FIRSTDATE_YR/ MM

In [27]:
categorical = categorical.drop(['CLUSTER','ODATEW_YR', 'ODATEW_MM','DOB_YR','DOB_MM', 'MINRDATE_YR', 'MAXRDATE_YR','LASTDATE_YR', 'FIRSTDATE_YR', 'FIRSTDATE_MM'], axis = 1)


In [28]:
categorical.dtypes

STATE          object
HOMEOWNR       object
GENDER         object
DATASRCE        int64
RFA_2R         object
RFA_2A         object
GEOCODE2       object
DOMAIN_A       object
DOMAIN_B        int64
MINRDATE_MM     int64
MAXRDATE_MM     int64
LASTDATE_MM     int64
dtype: object

In [29]:
categorical.select_dtypes(include='int')


,DATASRCE,DOMAIN_B,MINRDATE_MM,MAXRDATE_MM,LASTDATE_MM
0,3,2,8,2,12
1,3,1,10,12,12
2,3,2,11,7,12
3,3,2,11,11,12
4,3,2,10,1,1
...,...,...,...,...,...
95407,3,2,2,2,2
95408,3,1,3,3,3
95409,3,3,3,1,10
95410,2,1,11,8,1


In [30]:
convert_type_list = ['MINRDATE_MM','MAXRDATE_MM','LASTDATE_MM','DOMAIN_B','DATASRCE']

In [31]:
for item in convert_type_list:
    categorical[item] = categorical[item].astype('object')
    

In [32]:
categorical.dtypes

STATE          object
HOMEOWNR       object
GENDER         object
DATASRCE       object
RFA_2R         object
RFA_2A         object
GEOCODE2       object
DOMAIN_A       object
DOMAIN_B       object
MINRDATE_MM    object
MAXRDATE_MM    object
LASTDATE_MM    object
dtype: object

In [33]:
cat_clean = categorical

In [34]:
cat_clean

,STATE,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,MINRDATE_MM,MAXRDATE_MM,LASTDATE_MM
0,IL,H,F,3,L,E,C,T,2,8,2,12
1,CA,H,M,3,L,G,A,S,1,10,12,12
2,NC,U,M,3,L,E,C,R,2,11,7,12
3,CA,U,F,3,L,E,C,R,2,11,11,12
4,FL,H,F,3,L,F,A,S,2,10,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
95407,other,H,M,3,L,G,C,C,2,2,2,2
95408,TX,H,M,3,L,F,A,C,1,3,3,3
95409,MI,H,M,3,L,E,B,C,3,3,1,10
95410,CA,H,F,2,L,F,A,C,1,11,8,1


In [35]:
data = pd.concat((cat_clean,num_clean),axis=1)
data.head()

,STATE,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,MINRDATE_MM,MAXRDATE_MM,LASTDATE_MM,AGE,INCOME,WEALTH1,VIETVETS,WWIIVETS,WEALTH2,POP90C1,POP90C2,POP90C3,ETH1,ETH2,HHN3,DW1,DW2,DW4,DW5,DW6,HV1,HV2,HV3,HV4,HU1,HU2,HU5,HHD2,HHD3,HHD5,ETHC2,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR2,HUPA2,HUPA3,HUPA6,RP1,RP2,RP3,RP4,MSA,ADI,IC6,HHAS3,MC1,MC2,PEC2,TPE13,LFC2,LFC4,LFC6,LFC7,LFC8,LFC9,VC1,VC3,POBC2,LSC1,VOC2,HC2,HC4,HC5,HC6,HC7,HC8,HC11,HC13,HC17,HC18,HC19,MHUC1,MHUC2,CARDPROM,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2
0,IL,H,F,3,L,E,C,T,2,8,2,12,60.000000,5,9,34,18,5,0,35,65,92,1,49,97,95,2,0,0,479,635,3,2,86,14,7,80,70,84,50,2,7,13,27,47,0,61,0,0,0,2,5,17,73,0.0,177.0,13,51,46,54,13,45,64,64,62,53,99,0,34,18,74,88,77,31,14,14,31,54,46,90,10,33,65,40,6,2,27,95515,0,4,39
1,CA,H,M,3,L,G,A,S,1,10,12,12,46.000000,6,9,55,11,9,99,0,0,67,0,75,99,99,0,0,0,5468,5218,12,10,96,4,9,94,88,95,44,97,98,98,98,99,94,83,0,0,0,91,91,91,94,4480.0,13.0,2,66,56,44,2,71,83,81,64,57,99,99,55,11,39,50,92,10,26,56,97,99,0,96,4,99,0,99,20,4,12,148535,0,2,1
2,NC,U,M,3,L,E,C,R,2,11,7,12,61.611649,3,1,29,33,1,0,2,98,96,2,44,69,69,6,5,3,497,546,2,1,78,22,18,76,65,86,60,0,1,6,18,50,0,36,2,24,3,0,2,9,44,0.0,281.0,32,31,38,62,21,61,73,70,78,62,82,99,28,32,84,96,65,22,12,23,50,69,31,0,35,22,77,17,9,2,26,15078,1,4,60
3,CA,U,F,3,L,E,C,R,2,11,11,12,70.000000,1,4,14,31,0,0,8,92,61,0,50,85,83,4,1,0,1000,1263,2,1,48,52,6,73,61,84,36,10,25,50,69,92,10,42,0,9,0,1,8,17,34,9340.0,67.0,27,26,46,54,17,43,81,68,33,31,0,99,14,31,67,56,43,38,10,19,39,45,55,45,17,23,77,22,16,2,27,172556,1,4,41
4,FL,H,F,3,L,F,A,S,2,10,1,1,78.000000,3,2,9,53,9,99,0,0,2,98,57,99,98,0,0,0,576,594,4,3,90,10,0,82,49,92,1,0,1,2,16,67,0,45,0,0,0,25,58,74,83,5000.0,127.0,24,11,20,80,1,76,73,65,80,31,81,99,9,53,65,99,45,34,1,3,6,33,67,9,72,99,1,21,6,2,43,7112,1,2,26


In [36]:
#extract the correct target from targets dataframe

In [37]:
targets = targets.drop(columns=['TARGET_D'], axis=1)

In [38]:
y = targets['TARGET_B']

In [39]:
X = data

In [40]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [41]:
numericals_train = X_train.select_dtypes(np.number)
numericals_test = X_test.select_dtypes(np.number)
numericals_train.head()

,AGE,INCOME,WEALTH1,VIETVETS,WWIIVETS,WEALTH2,POP90C1,POP90C2,POP90C3,ETH1,ETH2,HHN3,DW1,DW2,DW4,DW5,DW6,HV1,HV2,HV3,HV4,HU1,HU2,HU5,HHD2,HHD3,HHD5,ETHC2,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR2,HUPA2,HUPA3,HUPA6,RP1,RP2,RP3,RP4,MSA,ADI,IC6,HHAS3,MC1,MC2,PEC2,TPE13,LFC2,LFC4,LFC6,LFC7,LFC8,LFC9,VC1,VC3,POBC2,LSC1,VOC2,HC2,HC4,HC5,HC6,HC7,HC8,HC11,HC13,HC17,HC18,HC19,MHUC1,MHUC2,CARDPROM,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2
4398,66.000000,4,0,15,48,9,0,95,5,71,28,38,77,75,15,11,9,464,518,2,2,63,37,12,66,49,78,33,0,1,6,16,44,0,39,6,7,8,1,5,19,51,0.0,247.0,45,31,44,56,9,32,65,57,75,37,82,99,15,48,83,95,45,23,4,15,48,73,27,54,34,99,0,90,9,2,12,118666,1,4,61
90538,61.611649,5,9,33,33,8,0,85,15,97,0,37,64,61,27,21,10,655,709,3,3,65,35,19,65,52,80,59,1,3,11,32,80,0,47,6,8,9,4,13,45,78,0.0,297.0,24,55,51,49,4,33,75,71,79,58,77,74,30,35,60,97,59,27,7,16,41,57,43,71,17,87,12,86,10,3,33,87263,0,1,37
16147,44.000000,5,4,39,40,9,0,99,0,99,0,39,67,66,32,13,10,453,493,3,3,66,34,9,66,56,83,56,0,0,1,6,37,0,56,6,0,10,0,2,36,86,0.0,111.0,19,61,40,60,14,46,85,84,76,64,99,0,39,40,85,96,56,51,0,0,0,11,89,61,2,99,0,99,5,3,8,79939,1,1,39
65008,43.000000,6,9,37,11,9,0,0,99,98,1,63,99,99,0,0,0,988,1055,9,9,99,1,17,96,92,98,70,1,5,48,92,99,0,92,0,0,0,99,99,99,99,4520.0,209.0,2,63,51,49,70,77,82,78,44,40,0,0,37,11,52,96,95,15,20,26,91,99,0,41,43,99,0,0,9,3,32,56284,0,1,15
52515,83.000000,5,4,26,51,4,99,0,0,99,0,43,93,90,5,4,4,762,814,3,3,90,10,0,81,73,91,57,0,4,20,52,88,0,69,3,2,3,5,13,54,85,0.0,233.0,9,55,29,71,2,46,73,69,43,41,0,99,26,51,60,98,81,36,0,2,19,39,61,2,88,99,0,58,6,1,31,182962,1,1,37


In [42]:
transformer = MinMaxScaler().fit(numericals_train)
numericals_train_standardized = transformer.transform(numericals_train)
numericals_test_standardized = transformer.transform(numericals_test)

In [43]:
pd.DataFrame(numericals_train_standardized).head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78
0,0.670103,0.500000,0.000000,0.151515,0.484848,1.000000,0.0,0.959596,0.050505,0.717172,0.282828,0.383838,0.777778,0.757576,0.151515,0.111111,0.090909,0.077333,0.086333,0.153846,0.153846,0.636364,0.373737,0.121212,0.666667,0.494949,0.787879,0.333333,0.000000,0.010101,0.060606,0.161616,0.444444,0.0,0.393939,0.060606,0.070707,0.080808,0.010101,0.050505,0.191919,0.515152,0.000000,0.379416,0.454545,0.313131,0.444444,0.565657,0.090909,0.323232,0.656566,0.575758,0.757576,0.373737,0.828283,1.000000,0.151515,0.484848,0.838384,0.959596,0.454545,0.442308,0.040404,0.151515,0.484848,0.737374,0.272727,0.545455,0.343434,1.000000,0.000000,0.909091,0.428571,0.4,0.183333,0.618762,1.0,1.0,0.983607
1,0.624862,0.666667,1.000000,0.333333,0.333333,0.888889,0.0,0.858586,0.151515,0.979798,0.000000,0.373737,0.646465,0.616162,0.272727,0.212121,0.101010,0.109167,0.118167,0.230769,0.230769,0.656566,0.353535,0.191919,0.656566,0.525253,0.808081,0.595960,0.010101,0.030303,0.111111,0.323232,0.808081,0.0,0.474747,0.060606,0.080808,0.090909,0.040404,0.131313,0.454545,0.787879,0.000000,0.456221,0.242424,0.555556,0.515152,0.494949,0.040404,0.333333,0.757576,0.717172,0.797980,0.585859,0.777778,0.747475,0.303030,0.353535,0.606061,0.979798,0.595960,0.519231,0.070707,0.161616,0.414141,0.575758,0.434343,0.717172,0.171717,0.878788,0.121212,0.868687,0.476190,0.6,0.533333,0.455016,0.0,0.0,0.590164
2,0.443299,0.666667,0.444444,0.393939,0.404040,1.000000,0.0,1.000000,0.000000,1.000000,0.000000,0.393939,0.676768,0.666667,0.323232,0.131313,0.101010,0.075500,0.082167,0.230769,0.230769,0.666667,0.343434,0.090909,0.666667,0.565657,0.838384,0.565657,0.000000,0.000000,0.010101,0.060606,0.373737,0.0,0.565657,0.060606,0.000000,0.101010,0.000000,0.020202,0.363636,0.868687,0.000000,0.170507,0.191919,0.616162,0.404040,0.606061,0.141414,0.464646,0.858586,0.848485,0.767677,0.646465,1.000000,0.000000,0.393939,0.404040,0.858586,0.969697,0.565657,0.980769,0.000000,0.000000,0.000000,0.111111,0.898990,0.616162,0.020202,1.000000,0.000000,1.000000,0.238095,0.6,0.116667,0.416826,1.0,0.0,0.622951
3,0.432990,0.833333,1.000000,0.373737,0.111111,1.000000,0.0,0.000000,1.000000,0.989899,0.010101,0.636364,1.000000,1.000000,0.000000,0.000000,0.000000,0.164667,0.175833,0.692308,0.692308,1.000000,0.010101,0.171717,0.969697,0.929293,0.989899,0.707071,0.010101,0.050505,0.484848,0.929293,1.000000,0.0,0.929293,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.482906,0.321045,0.020202,0.636364,0.515152,0.494949,0.707071,0.777778,0.828283,0.787879,0.444444,0.404040,0.000000,0.000000,0.373737,0.111111,0.525253,0.969697,0.959596,0.288462,0.202020,0.262626,0.919192,1.000000,0.000000,0.414141,0.434343,1.000000,0.000000,0.000000,0.428571,0.6,0.516667,0.293480,0.0,0.0,0.229508
4,0.845361,0.666667,0.444444,0.262626,0.515152,0.444444,1.0,0.000000,0.000000,1.000000,0.000000,0.434343,0.939394,0.909091,0.050505,0.040404,0.040404,0.127000,0.135667,0.230769,0.230769,0.909091,0.101010,0.000000,0.818182,0.737374,0.919192,0.575758,0.000000,0.040404,0.202020,0.525253,0.888889,0.0,0.696970,0.030303,0.020202,0.030303,0.050505,0.131313,0.545455,0.858586,0.000000,0.357911,0.090909,0.555556,0.292929,0.717172,0.020202,0.464646,0.737374,0.696970,0.434343,0.414141,0.000000,1.000000,0.262626,0.515152,0.606061,0.989899,0.818182,0.692308,0.000000,0.020202,0.191919,0.393939,0.616162,0.020202,0.888889,1.000000,0.000000,0.585859,0.285714,0.2,0.500000,0.954025,1.0,0.0,0.590164


In [44]:
categoricals_train= X_train.select_dtypes(object)
categoricals_test= X_test.select_dtypes(object)
categoricals_train.head()

onehot_train = categoricals_train.drop(['DOMAIN_B'], axis = 1)
onehot_test = categoricals_test.drop(['DOMAIN_B'], axis = 1)
onehot_train.head()

,STATE,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,MINRDATE_MM,MAXRDATE_MM,LASTDATE_MM
4398,TX,H,F,3,L,F,D,R,10,7,3
90538,other,H,F,3,L,F,D,T,9,8,12
16147,WI,H,M,3,L,F,C,T,1,12,12
65008,other,H,F,3,L,F,B,T,3,1,1
52515,WA,H,M,3,L,E,C,C,11,9,11


In [45]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='error', drop='first').fit(onehot_train)
onehot_train_encoded = encoder.transform(onehot_train).toarray()
onehot_test_encoded = encoder.transform(onehot_test).toarray()

In [46]:
ordinal_train = categoricals_train[['DOMAIN_B']]
ordinal_test = categoricals_test[['DOMAIN_B']]
ordinal_train.head()

,DOMAIN_B
4398,2
90538,2
16147,2
65008,1
52515,2


In [47]:
X_train_processed = np.concatenate((numericals_train_standardized, onehot_train_encoded, ordinal_train), axis = 1)
X_test_processed = np.concatenate((numericals_test_standardized, onehot_test_encoded, ordinal_test), axis = 1)

In [48]:
pd.DataFrame(X_train_processed)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138
0,0.670103,0.5,0.0,0.151515,0.484848,1.0,0.0,0.959596,0.050505,0.717172,0.282828,0.383838,0.777778,0.757576,0.151515,0.111111,0.090909,0.077333,0.086333,0.153846,0.153846,0.636364,0.373737,0.121212,0.666667,0.494949,0.787879,0.333333,0.0,0.010101,0.060606,0.161616,0.444444,0.0,0.393939,0.060606,0.070707,0.080808,0.010101,0.050505,0.191919,0.515152,0.0,0.379416,0.454545,0.313131,0.444444,0.565657,0.090909,0.323232,0.656566,0.575758,0.757576,0.373737,0.828283,1.0,0.151515,0.484848,0.838384,0.959596,0.454545,0.442308,0.040404,0.151515,0.484848,0.737374,0.272727,0.545455,0.343434,1.0,0.0,0.909091,0.428571,0.4,0.183333,0.618762,1.0,1.0,0.983607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
1,0.624862,0.666667,1.0,0.333333,0.333333,0.888889,0.0,0.858586,0.151515,0.979798,0.0,0.373737,0.646465,0.616162,0.272727,0.212121,0.10101,0.109167,0.118167,0.230769,0.230769,0.656566,0.353535,0.191919,0.656566,0.525253,0.808081,0.59596,0.010101,0.030303,0.111111,0.323232,0.808081,0.0,0.474747,0.060606,0.080808,0.090909,0.040404,0.131313,0.454545,0.787879,0.0,0.456221,0.242424,0.555556,0.515152,0.494949,0.040404,0.333333,0.757576,0.717172,0.79798,0.585859,0.777778,0.747475,0.30303,0.353535,0.606061,0.979798,0.59596,0.519231,0.070707,0.161616,0.414141,0.575758,0.434343,0.717172,0.171717,0.878788,0.121212,0.868687,0.47619,0.6,0.533333,0.455016,0.0,0.0,0.590164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2
2,0.443299,0.666667,0.444444,0.393939,0.40404,1.0,0.0,1.0,0.0,1.0,0.0,0.393939,0.676768,0.666667,0.323232,0.131313,0.10101,0.0755,0.082167,0.230769,0.230769,0.666667,0.343434,0.090909,0.666667,0.565657,0.838384,0.565657,0.0,0.0,0.010101,0.060606,0.373737,0.0,0.565657,0.060606,0.0,0.10101,0.0,0.020202,0.363636,0.868687,0.0,0.170507,0.191919,0.616162,0.40404,0.606061,0.141414,0.464646,0.858586,0.848485,0.767677,0.646465,1.0,0.0,0.393939,0.40404,0.858586,0.969697,0.565657,0.980769,0.0,0.0,0.0,0.111111,0.89899,0.616162,0.020202,1.0,0.0,1.0,0.238095,0.6,0.116667,0.416826,1.0,0.0,0.622951,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2
3,0.43299,0.833333,1.0,0.373737,0.111111,1.0,0.0,0.0,1.0,0.989899,0.010101,0.636364,1.0,1.0,0.0,0.0,0.0,0.164667,0.175833,0.692308,0.692308,1.0,0.010101,0.171717,0.969697,0.929293,0.989899,0.707071,0.010101,0.050505,0.484848,0.929293,1.0,0.0,0.929293,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.482906,0.321045,0.020202,0.636364,0.515152,0.494949,0.707071,0.777778,0.828283,0.787879,0.444444,0.40404,0.0,0.0,0.373737,0.111111,0.525253,0.969697,0.959596,0.288462,0.20202,0.262626,0.919192,1.0,0.0,0.414141,0.434343,1.0,0.0,0.0,0.428571,0.6,0.516667,0.29348,0.0,0.0,0.229508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.845361,0.666667,0.444444,0.262626,0.515152,0.444444,1.0,0.0,0.0,1.0,0.0,0.434343,0.939394,0.909091,0.050505,0.040404,0.040404,0.127,0

In [49]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression(random_state=0, solver='lbfgs') # wanted to try a different solver
LR.fit(X_train_processed, y_train) 
LR.score(X_test_processed, y_test)   

0.9474279964784303

In [50]:
pred = LR.predict(X_test_processed)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,pred)

array([[22599,     0],
       [ 1254,     0]])

In [51]:
print(y_test.value_counts())


0    22599
1     1254
Name: TARGET_B, dtype: int64


In [52]:
#we still have imbalance

In [53]:
X_train_imbalance = pd.DataFrame(X_train_processed)


In [54]:
X_train_imbalance.isna().sum()
# making sure there are no nulls

0      0
1      0
2      0
3      0
4      0
      ..
134    0
135    0
136    0
137    0
138    0
Length: 139, dtype: int64

In [55]:
display(X_train_imbalance.shape)
display(y_train.shape)

(71559, 139)

(71559,)

In [56]:
display(X_train_imbalance.head())
pd.DataFrame(y_train).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138
0,0.670103,0.5,0.0,0.151515,0.484848,1.0,0.0,0.959596,0.050505,0.717172,0.282828,0.383838,0.777778,0.757576,0.151515,0.111111,0.090909,0.077333,0.086333,0.153846,0.153846,0.636364,0.373737,0.121212,0.666667,0.494949,0.787879,0.333333,0.0,0.010101,0.060606,0.161616,0.444444,0.0,0.393939,0.060606,0.070707,0.080808,0.010101,0.050505,0.191919,0.515152,0.0,0.379416,0.454545,0.313131,0.444444,0.565657,0.090909,0.323232,0.656566,0.575758,0.757576,0.373737,0.828283,1.0,0.151515,0.484848,0.838384,0.959596,0.454545,0.442308,0.040404,0.151515,0.484848,0.737374,0.272727,0.545455,0.343434,1.0,0.0,0.909091,0.428571,0.4,0.183333,0.618762,1.0,1.0,0.983607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
1,0.624862,0.666667,1.0,0.333333,0.333333,0.888889,0.0,0.858586,0.151515,0.979798,0.0,0.373737,0.646465,0.616162,0.272727,0.212121,0.10101,0.109167,0.118167,0.230769,0.230769,0.656566,0.353535,0.191919,0.656566,0.525253,0.808081,0.59596,0.010101,0.030303,0.111111,0.323232,0.808081,0.0,0.474747,0.060606,0.080808,0.090909,0.040404,0.131313,0.454545,0.787879,0.0,0.456221,0.242424,0.555556,0.515152,0.494949,0.040404,0.333333,0.757576,0.717172,0.79798,0.585859,0.777778,0.747475,0.30303,0.353535,0.606061,0.979798,0.59596,0.519231,0.070707,0.161616,0.414141,0.575758,0.434343,0.717172,0.171717,0.878788,0.121212,0.868687,0.47619,0.6,0.533333,0.455016,0.0,0.0,0.590164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2
2,0.443299,0.666667,0.444444,0.393939,0.40404,1.0,0.0,1.0,0.0,1.0,0.0,0.393939,0.676768,0.666667,0.323232,0.131313,0.10101,0.0755,0.082167,0.230769,0.230769,0.666667,0.343434,0.090909,0.666667,0.565657,0.838384,0.565657,0.0,0.0,0.010101,0.060606,0.373737,0.0,0.565657,0.060606,0.0,0.10101,0.0,0.020202,0.363636,0.868687,0.0,0.170507,0.191919,0.616162,0.40404,0.606061,0.141414,0.464646,0.858586,0.848485,0.767677,0.646465,1.0,0.0,0.393939,0.40404,0.858586,0.969697,0.565657,0.980769,0.0,0.0,0.0,0.111111,0.89899,0.616162,0.020202,1.0,0.0,1.0,0.238095,0.6,0.116667,0.416826,1.0,0.0,0.622951,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2
3,0.43299,0.833333,1.0,0.373737,0.111111,1.0,0.0,0.0,1.0,0.989899,0.010101,0.636364,1.0,1.0,0.0,0.0,0.0,0.164667,0.175833,0.692308,0.692308,1.0,0.010101,0.171717,0.969697,0.929293,0.989899,0.707071,0.010101,0.050505,0.484848,0.929293,1.0,0.0,0.929293,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.482906,0.321045,0.020202,0.636364,0.515152,0.494949,0.707071,0.777778,0.828283,0.787879,0.444444,0.40404,0.0,0.0,0.373737,0.111111,0.525253,0.969697,0.959596,0.288462,0.20202,0.262626,0.919192,1.0,0.0,0.414141,0.434343,1.0,0.0,0.0,0.428571,0.6,0.516667,0.29348,0.0,0.0,0.229508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.845361,0.666667,0.444444,0.262626,0.515152,0.444444,1.0,0.0,0.0,1.0,0.0,0.434343,0.939394,0.909091,0.050505,0.040404,0.040404,0.127,0

,TARGET_B
4398,0
90538,0
16147,0
65008,0
52515,0


In [57]:
y_train = y_train.reset_index(drop=True)
y_train


0        0
1        0
2        0
3        0
4        0
        ..
71554    0
71555    0
71556    1
71557    0
71558    0
Name: TARGET_B, Length: 71559, dtype: int64

In [58]:
#concat the training data

In [59]:
trainingdata = pd.concat((X_train_imbalance, y_train), axis = 1)
trainingdata.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,TARGET_B
0,0.670103,0.5,0.0,0.151515,0.484848,1.0,0.0,0.959596,0.050505,0.717172,0.282828,0.383838,0.777778,0.757576,0.151515,0.111111,0.090909,0.077333,0.086333,0.153846,0.153846,0.636364,0.373737,0.121212,0.666667,0.494949,0.787879,0.333333,0.0,0.010101,0.060606,0.161616,0.444444,0.0,0.393939,0.060606,0.070707,0.080808,0.010101,0.050505,0.191919,0.515152,0.0,0.379416,0.454545,0.313131,0.444444,0.565657,0.090909,0.323232,0.656566,0.575758,0.757576,0.373737,0.828283,1.0,0.151515,0.484848,0.838384,0.959596,0.454545,0.442308,0.040404,0.151515,0.484848,0.737374,0.272727,0.545455,0.343434,1.0,0.0,0.909091,0.428571,0.4,0.183333,0.618762,1.0,1.0,0.983607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0
1,0.624862,0.666667,1.0,0.333333,0.333333,0.888889,0.0,0.858586,0.151515,0.979798,0.0,0.373737,0.646465,0.616162,0.272727,0.212121,0.10101,0.109167,0.118167,0.230769,0.230769,0.656566,0.353535,0.191919,0.656566,0.525253,0.808081,0.59596,0.010101,0.030303,0.111111,0.323232,0.808081,0.0,0.474747,0.060606,0.080808,0.090909,0.040404,0.131313,0.454545,0.787879,0.0,0.456221,0.242424,0.555556,0.515152,0.494949,0.040404,0.333333,0.757576,0.717172,0.79798,0.585859,0.777778,0.747475,0.30303,0.353535,0.606061,0.979798,0.59596,0.519231,0.070707,0.161616,0.414141,0.575758,0.434343,0.717172,0.171717,0.878788,0.121212,0.868687,0.47619,0.6,0.533333,0.455016,0.0,0.0,0.590164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0
2,0.443299,0.666667,0.444444,0.393939,0.40404,1.0,0.0,1.0,0.0,1.0,0.0,0.393939,0.676768,0.666667,0.323232,0.131313,0.10101,0.0755,0.082167,0.230769,0.230769,0.666667,0.343434,0.090909,0.666667,0.565657,0.838384,0.565657,0.0,0.0,0.010101,0.060606,0.373737,0.0,0.565657,0.060606,0.0,0.10101,0.0,0.020202,0.363636,0.868687,0.0,0.170507,0.191919,0.616162,0.40404,0.606061,0.141414,0.464646,0.858586,0.848485,0.767677,0.646465,1.0,0.0,0.393939,0.40404,0.858586,0.969697,0.565657,0.980769,0.0,0.0,0.0,0.111111,0.89899,0.616162,0.020202,1.0,0.0,1.0,0.238095,0.6,0.116667,0.416826,1.0,0.0,0.622951,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0
3,0.43299,0.833333,1.0,0.373737,0.111111,1.0,0.0,0.0,1.0,0.989899,0.010101,0.636364,1.0,1.0,0.0,0.0,0.0,0.164667,0.175833,0.692308,0.692308,1.0,0.010101,0.171717,0.969697,0.929293,0.989899,0.707071,0.010101,0.050505,0.484848,0.929293,1.0,0.0,0.929293,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.482906,0.321045,0.020202,0.636364,0.515152,0.494949,0.707071,0.777778,0.828283,0.787879,0.444444,0.40404,0.0,0.0,0.373737,0.111111,0.525253,0.969697,0.959596,0.288462,0.20202,0.262626,0.919192,1.0,0.0,0.414141,0.434343,1.0,0.0,0.0,0.428571,0.6,0.516667,0.29348,0.0,0.0,0.229508,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
4,0.845361,0.666667,0.444444,0.262626,0.515152,0.444444,1.0,0.0,0.0,1.0,0.0,0.434343,0.939394,0.909091,0.050505,0.040404

In [60]:
trainingdata['TARGET_B'].value_counts()


0    67970
1     3589
Name: TARGET_B, dtype: int64

In [61]:
from sklearn.utils import resample

category_0 = trainingdata[trainingdata['TARGET_B'] == 0]
category_1 = trainingdata[trainingdata['TARGET_B'] == 1]

In [62]:
#Downsampling

In [63]:
category_0_undersampled = resample(category_0, 
                                   replace=False, 
                                   n_samples = len(category_1)) 

In [64]:
print(category_0_undersampled.shape)
print(category_1.shape)

(3589, 140)
(3589, 140)


In [65]:
data_downsampled = pd.concat([category_0_undersampled, category_1], axis=0) 


In [66]:
data_downsampled_test = data_downsampled.dropna()


In [67]:
data_downsampled_test['TARGET_B'].value_counts()


0    3589
1    3589
Name: TARGET_B, dtype: int64

In [68]:
X_train_down_test_data = data_downsampled_test.drop(['TARGET_B'], axis = 1)
y_train_down_test_data = data_downsampled_test['TARGET_B']

In [69]:
display(X_train_down_test_data)
pd.DataFrame(y_train_down_test_data)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138
10060,0.42268,0.333333,1.0,0.555556,0.212121,1.0,1.0,0.0,0.0,0.959596,0.010101,0.626263,1.0,1.0,0.0,0.0,0.0,0.138167,0.136167,0.461538,0.461538,0.929293,0.080808,0.131313,0.858586,0.767677,0.929293,0.626263,0.0,0.010101,0.060606,0.707071,1.0,0.0,0.787879,0.0,0.0,0.0,0.878788,1.0,1.0,1.0,0.910256,0.425499,0.030303,0.444444,0.353535,0.656566,0.040404,0.767677,0.777778,0.777778,0.676768,0.515152,1.0,1.0,0.555556,0.212121,0.373737,0.959596,0.868687,0.288462,0.0,0.191919,1.0,1.0,0.0,0.323232,0.646465,1.0,0.0,1.0,0.333333,0.4,0.066667,0.72197,0.0,0.0,0.311475,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1
59721,0.536082,0.0,1.0,0.444444,0.0,0.555556,0.0,0.0,1.0,0.757576,0.252525,0.59596,0.636364,0.626263,0.0,0.0,0.0,0.091333,0.113833,0.153846,0.153846,0.89899,0.111111,0.070707,0.818182,0.666667,0.878788,0.525253,0.020202,0.020202,0.212121,0.313131,0.575758,0.010101,0.474747,0.0,0.363636,0.0,0.0,0.141414,0.313131,0.535354,0.0,0.554531,0.131313,0.20202,0.353535,0.656566,0.676768,0.787879,0.808081,0.757576,0.676768,0.494949,1.0,1.0,0.444444,0.0,0.626263,0.959596,0.626263,0.288462,0.20202,0.414141,0.727273,0.949495,0.060606,0.131313,0.636364,0.727273,0.282828,0.181818,0.285714,0.4,0.25,0.117996,0.0,0.333333,0.704918,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
54617,0.824742,0.5,1.0,0.323232,0.333333,0.777778,1.0,0.0,0.0,0.909091,0.010101,0.525253,0.808081,0.737374,0.191919,0.191919,0.171717,0.461667,0.4915,0.615385,0.615385,0.727273,0.282828,0.030303,0.767677,0.646465,0.838384,0.59596,0.888889,0.979798,1.0,1.0,1.0,0.373737,0.646465,0.141414,0.0,0.151515,0.888889,0.919192,0.929293,0.949495,0.933226,0.019969,0.10101,0.545455,0.505051,0.505051,0.393939,0.545455,0.848485,0.818182,0.656566,0.515152,0.848485,0.929293,0.30303,0.272727,0.444444,0.858586,0.767677,0.403846,0.050505,0.141414,0.484848,0.919192,0.090909,0.868687,0.121212,1.0,0.0,1.0,0.47619,0.8,0.45,0.021535,0.0,0.333333,0.163934,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
29611,0.391753,0.666667,0.555556,0.414141,0.20202,1.0,0.606061,0.232323,0.171717,0.979798,0.0,0.525253,0.636364,0.626263,0.121212,0.10101,0.060606,0.161,0.202167,0.307692,0.307692,0.767677,0.242424,0.191919,0.787879,0.646465,0.878788,0.606061,0.121212,0.222222,0.474747,0.727273,0.959596,0.040404,0.535354,0.040404,0.242424,0.050505,0.151515,0.393939,0.787879,0.919192,0.876068,0.16129,0.151515,0.40404,0.515152,0.494949,0.565657,0.757576,0.818182,0.777778,0.636364,0.484848,0.767677,0.888889,0.40404,0.232323,0.575758,0.969697,0.747475,0.307692,0.151515,0.333333,0.707071,0.848485,0.161616,0.151515,0.575758,0.939394,0.050505,0.40404,0.47619,0.6,0.083333,0.948263,1.0,0.333333,0.344262,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
46340,0.319588,0.333333,1.0,0.525253,0.171717,1.0,0.0,1.0,0.0,0.949495,0.050505,0.525253,0.777778,

,TARGET_B
10060,0
59721,0
54617,0
29611,0
46340,0
...,...
71491,1
71506,1
71530,1
71531,1


In [80]:
LR_downsampled = LogisticRegression(random_state=0, solver='lbfgs')
LR_downsampled.fit(X_train_down_test_data, y_train_down_test_data)
LR_downsampled.score(X_test_processed, y_test)

0.591539848237119

In [81]:
pred_down = LR_downsampled.predict(X_test_processed)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,pred_down)

array([[13423,  9176],
       [  567,   687]])

In [72]:
category_1_oversampled = resample(category_1, 
                                  replace=True, 
                                  n_samples = len(category_0)) 

In [73]:
print(category_0.shape)
print(category_1_oversampled.shape)

(67970, 140)
(67970, 140)


In [74]:
data_upsampled = pd.concat([category_0, category_1_oversampled], axis=0)


In [75]:
data_upsampled['TARGET_B'].value_counts()


0    67970
1    67970
Name: TARGET_B, dtype: int64

In [76]:
X_train_up = data_upsampled.drop(['TARGET_B'], axis = 1)
y_train_up = data_upsampled['TARGET_B']

In [82]:
LR_upsampled = LogisticRegression(random_state=0, solver='lbfgs')
LR_upsampled.fit(X_train_up, y_train_up)
LR_upsampled.score(X_test_processed, y_test)

0.6139269693539596

In [83]:
pred_up = LR_upsampled.predict(X_test_processed)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,pred_up)

array([[13964,  8635],
       [  574,   680]])

In [79]:
# we can see how downsampling and upsamlpling both correct the confusion matrix significantly